In [80]:
from dotenv import load_dotenv,find_dotenv
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate

from langchain_core.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticToolsParser
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnablePassthrough

In [2]:
load_dotenv(dotenv_path=find_dotenv(filename="D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

In [70]:
@tool
def add(a:int,b:int) -> int:
    """
        Adds a and b
    Args:
        a: first int
        b: second int
    """
    return a+b

In [71]:
@tool
def multiply(a:int,b:int) -> int:
    """
        Multiplies a and b
    Args:
        a: first int
        b: second int
    """
    return a*b

In [72]:
tools=[add,multiply]

In [48]:
class add(BaseModel):
    """
        Add two integers togather
    """
    a: int=Field(default=...,description="First Integer")
    b: int=Field(default=...,description="Second Integer")

In [49]:
class multiply(BaseModel):
    """
        Multiplies two integers togather
    """
    a: int=Field(default=...,description="First Integer")
    b: int=Field(default=...,description="Second Integer")

In [50]:
...

Ellipsis

In [51]:
tools=[add,multiply]

In [52]:
llmWithTools=llmGemini.bind_tools(
    tools=tools
)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [53]:
llmWithAlwaysCall=llmGemini.bind_tools(tools=tools,
                                  tool_choice="any"  # atleast call one tool, replace with multiply or add for preference
                                 )

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [54]:
llmWithMultiply=llmGemini.bind_tools(tools=tools,
                                  tool_choice="multiply"  # atleast call one tool, replace with multiply or add for preference
                                 )

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [55]:
query="What is  3*12? Also, what is 11+49?"

In [56]:
llmWithTools.invoke(input=query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3.0, 'b': 12.0},
  'id': 'db5ff35f-775f-4b5a-bb79-eb4cb9a939d6',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11.0, 'b': 49.0},
  'id': 'f46a9c85-89a9-4b88-ae43-5cb765a69ae4',
  'type': 'tool_call'}]

In [57]:
chain=llmWithTools|PydanticToolsParser(tools=[multiply,add])

In [58]:
chain.invoke(input=query)

[multiply(a=3, b=12), add(a=11, b=49)]

<h3>Streaming</h3>

In [59]:
async for chunk in llmWithTools.astream(input=query):
    print(chunk.tool_call_chunks)

[{'name': 'multiply', 'args': '{"a": 3.0, "b": 12.0}', 'id': '488a4da9-2d86-4c98-ba87-5b22c84f30d4', 'index': None, 'type': 'tool_call_chunk'}]
[{'name': 'add', 'args': '{"a": 11.0, "b": 49.0}', 'id': '270efb5b-0b8d-41c6-88de-aaca68a412f0', 'index': None, 'type': 'tool_call_chunk'}]


In [60]:
# Accumulating Chunks
first=True
async for chunk in llmWithTools.astream(input=query):
    if first:
        gathered=chunk
        first=False
    else:
        gathered+=chunk

    print(gathered.tool_call_chunks)

[{'name': 'multiply', 'args': '{"a": 3.0, "b": 12.0}', 'id': '7f5bf406-1d26-4662-b23d-72783f076e4e', 'index': None, 'type': 'tool_call_chunk'}, {'name': 'add', 'args': '{"a": 11.0, "b": 49.0}', 'id': '878647a4-e7fa-450e-80e2-810e5f0adb6f', 'index': None, 'type': 'tool_call_chunk'}]


In [61]:
# Accumulating Chunks
first=True
async for chunk in llmWithTools.astream(input=query):
    if first:
        gathered=chunk
        first=False
    else:
        gathered+=chunk

    print(gathered.tool_calls)

[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '7cfdad35-c20f-4127-bf4c-cbf1011979e8', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '290c7bae-2375-484f-9421-b2378bbf545d', 'type': 'tool_call'}]


<h3>Passing Tool Outputs to Models</h3>

In [62]:
messages=[HumanMessage(query)]

In [63]:
messages

[HumanMessage(content='What is  3*12? Also, what is 11+49?', additional_kwargs={}, response_metadata={})]

In [64]:
aiMessage=llmWithTools.invoke(input=messages)
aiMessage

AIMessage(content='\n', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-72c80189-1976-4dab-8023-e539d9f9eb3c-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '4046785d-e924-40ca-a019-f88009737f9b', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '3cb55886-9c11-4ff1-bb66-5c95b7ee3971', 'type': 'tool_call'}], usage_metadata={'input_tokens': 113, 'output_tokens': 41, 'total_tokens': 154})

In [65]:
aiMessage.tool_calls

[{'name': 'multiply',
  'args': {'a': 3.0, 'b': 12.0},
  'id': '4046785d-e924-40ca-a019-f88009737f9b',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11.0, 'b': 49.0},
  'id': '3cb55886-9c11-4ff1-bb66-5c95b7ee3971',
  'type': 'tool_call'}]

In [66]:
messages.append(aiMessage)

In [67]:
messages

[HumanMessage(content='What is  3*12? Also, what is 11+49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='\n', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-72c80189-1976-4dab-8023-e539d9f9eb3c-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '4046785d-e924-40ca-a019-f88009737f9b', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '3cb55886-9c11-4ff1-bb66-5c95b7ee3971', 'ty

In [46]:
# Illustrations
ToolMessage(content=add.invoke(input= {'a': 11.0, 'b': 49.0}),tool_call_id='0ab77964-1afb-4f8c-82ea-74f55f5c0e12')

ToolMessage(content='60', tool_call_id='0ab77964-1afb-4f8c-82ea-74f55f5c0e12')

In [73]:
for toolCall in aiMessage.tool_calls:
    selectedTool={"add":add,"multiply":multiply}[toolCall['name'].lower()]
    toolOutput=selectedTool.invoke(input=toolCall['args'])
    messages.append(ToolMessage(content=toolOutput,tool_call_id=toolCall['id']))

In [74]:
messages

[HumanMessage(content='What is  3*12? Also, what is 11+49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='\n', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"a": 11.0, "b": 49.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-72c80189-1976-4dab-8023-e539d9f9eb3c-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 12.0}, 'id': '4046785d-e924-40ca-a019-f88009737f9b', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11.0, 'b': 49.0}, 'id': '3cb55886-9c11-4ff1-bb66-5c95b7ee3971', 'ty

<h3>Few Shot Prompting</h3>

In [78]:
examples=[
    HumanMessage(
        content="What's the product of 317253 and 128472 plus four",
        name="exampleUser"
    ),
    AIMessage(
        content="",
        name="exampleAssistant",
        tool_calls=[
            {"name":"multiply","args":{"x":317253,"y":128472},"id":"1"}
        ]
    ),
    ToolMessage(content="16505054784",tool_call_id="1"),
    AIMessage(
        content="",
        name="exampleAssistant",
        tool_calls=[
            {"name":"add","args":{"x":16505054784,"y":4},"id":"2"}
        ]
    ),
    ToolMessage(content="16505054784",tool_call_id="2"),
    AIMessage(
        content="The product of 317253 and 128472 plus four is 16505054784",
        name="exampleAssistant"
    ),
]

In [79]:
fewShotPromptTemplate=ChatPromptTemplate.from_messages(
    [
        ("system",""" 
                You are bad at math but are an expert at using a calculator. 
                Use past tool usage as an example of how to correctly use the tools.
        """),
        *examples,
        ("human","{query}")
    ]
)

In [81]:
chain=RunnablePassthrough()|fewShotPromptTemplate|llmWithTools

In [85]:
response=chain.invoke(input={'query':'What is 119 times 8 minus 20'})

In [86]:
response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 119.0, "b": 8.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-f2c1f71d-7c7a-491f-994b-8770e8b31647-0', tool_calls=[{'name': 'multiply', 'args': {'a': 119.0, 'b': 8.0}, 'id': '08828331-42cc-4b90-a7b6-6e8bb7b40358', 'type': 'tool_call'}], usage_metadata={'input_tokens': 252, 'output_tokens': 20, 'total_tokens': 272})

In [88]:
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 119.0, 'b': 8.0},
  'id': '08828331-42cc-4b90-a7b6-6e8bb7b40358',
  'type': 'tool_call'}]